In [1]:
import networkx as nx
import numpy as np

from Methods import *
from Evaluation import *
from DAG_generation import *

set_seed()

NOTEARS implementation description

Method for finding DAG structure learnign for linear models, by solving the optimization problem via a poximal quasi newton method (C.F algorithm 1 in the NOTEARS paper)

DAG utils. Functions for evaluating if a weighted adjacency matrix describes a DAG, for generating DAGs according to Erdos-Renyi, Barbarasi or bipartite schemes,

In [2]:


####################################### Ground truth graph and params generation ######################


def simulate_dag(d, s0, graph_type):
    """Simulate random DAG with some expected number of edges.

    Args:
        d (int): num of nodes
        s0 (int): expected num of edges
        graph_type (str): ER, SF, BP

    Returns:
        B (np.ndarray): [d, d] binary adj matrix of DAG
    """
    def _random_permutation(M): # Randomly permutes matrix M
        P = np.random.permutation(np.eye(M.shape[0]))
        return P.T @ M @ P

    def _random_acyclic_orientation(B_und): # Randomly permutes the adjacency matrix (induces an ordering on the vertices) and the lower triangulation enforces a topological ordering
        return np.tril(_random_permutation(B_und), k=-1)

    def _graph_to_adjmat(G): # converts a graph from the igraph library to its adjacency matrix (numpy array)
        return np.array(G.get_adjacency().data)

    if graph_type == 'ER':
        # Erdos-Renyi graph: the n,M model where we enforce M edges to be present in the graph in expectation
        G_und = ig.Graph.Erdos_Renyi(n=d, m=s0)
        B_und = _graph_to_adjmat(G_und)
        B = _random_acyclic_orientation(B_und) # Enforcing acyclicity
    elif graph_type == 'SF': #
        # Scale-free, Barabasi-Albert the more connections a node has, the more likely it is to have more edges
        G = ig.Graph.Barabasi(n=d, m=int(round(s0 / d)), directed=True)
        B = _graph_to_adjmat(G)
    elif graph_type == 'BP':
        # Bipartite, Sec 4.1 of (Gu, Fu, Zhou, 2018)
        top = int(0.2 * d)
        G = ig.Graph.Random_Bipartite(top, d - top, m=s0, directed=True, neimode=ig.OUT)
        B = _graph_to_adjmat(G)
    else:
        raise ValueError('unknown graph type')
    B_perm = _random_permutation(B)
    assert ig.Graph.Adjacency(B_perm.tolist()).is_dag()
    return B_perm


def simulate_parameter(B, w_ranges=((-2.0, -0.5), (0.5, 2.0))):
    """Simulate SEM parameters for a DAG.

    Args:
        B (np.ndarray): [d, d] binary adj matrix of DAG
        w_ranges (tuple): disjoint weight ranges

    Returns:
        W (np.ndarray): [d, d] weighted adj matrix of DAG
    """
    W = np.zeros(B.shape)
    S = np.random.randint(len(w_ranges), size=B.shape)  # which range
    for i, (low, high) in enumerate(w_ranges):
        U = np.random.uniform(low=low, high=high, size=B.shape)
        W += B * (S == i) * U
    return W




Simulate samples from either a linear on non linear structural equation model

In [3]:
def simulate_linear_sem(W, n, sem_type, noise_scale=None):
    """Simulate samples from linear SEM with specified type of noise.

    For uniform, noise z ~ uniform(-a, a), where a = noise_scale.

    Args:
        W (np.ndarray): [d, d] weighted adj matrix of DAG
        n (int): num of samples, n=inf mimics population risk
        sem_type (str): gauss, exp, gumbel, uniform, logistic, poisson
        noise_scale (np.ndarray): scale parameter of additive noise, default all ones

    Returns:
        X (np.ndarray): [n, d] sample matrix, [d, d] if n=inf
    """
    def _simulate_single_equation(X, w, scale):
        """X: [n, num of parents], w: [num of parents], x: [n]"""
        if sem_type == 'gauss':
            z = np.random.normal(scale=scale, size=n)
            x = X @ w + z
        elif sem_type == 'exp':
            z = np.random.exponential(scale=scale, size=n)
            x = X @ w + z
        elif sem_type == 'gumbel':
            z = np.random.gumbel(scale=scale, size=n)
            x = X @ w + z
        elif sem_type == 'uniform':
            z = np.random.uniform(low=-scale, high=scale, size=n)
            x = X @ w + z
        elif sem_type == 'logistic':
            x = np.random.binomial(1, sigmoid(X @ w)) * 1.0
        elif sem_type == 'poisson':
            x = np.random.poisson(np.exp(X @ w)) * 1.0
        else:
            raise ValueError('unknown sem type')
        return x

    d = W.shape[0]
    if noise_scale is None:
        scale_vec = np.ones(d)
    elif np.isscalar(noise_scale):
        scale_vec = noise_scale * np.ones(d)
    else:
        if len(noise_scale) != d:
            raise ValueError('noise scale must be a scalar or has length d')
        scale_vec = noise_scale
    if not is_dag(W):
        raise ValueError('W must be a DAG')
    if np.isinf(n):  # population risk for linear gauss SEM
        if sem_type == 'gauss':
            # make 1/d X'X = true cov
            X = np.sqrt(d) * np.diag(scale_vec) @ np.linalg.inv(np.eye(d) - W)
            return X
        else:
            raise ValueError('population risk not available')
    # empirical risk
    G = ig.Graph.Weighted_Adjacency(W.tolist())
    ordered_vertices = G.topological_sorting()
    assert len(ordered_vertices) == d
    X = np.zeros([n, d])
    for j in ordered_vertices:
        parents = G.neighbors(j, mode=ig.IN)
        X[:, j] = _simulate_single_equation(X[:, parents], W[parents, j], scale_vec[j])
    return X


def simulate_nonlinear_sem(B, n, sem_type, noise_scale=None):
    """Simulate samples from nonlinear SEM.

    Args:
        B (np.ndarray): [d, d] binary adj matrix of DAG
        n (int): num of samples
        sem_type (str): mlp, mim, gp, gp-add
        noise_scale (np.ndarray): scale parameter of additive noise, default all ones

    Returns:
        X (np.ndarray): [n, d] sample matrix
    """
    def _simulate_single_equation(X, scale):
        """X: [n, num of parents], x: [n]"""
        z = np.random.normal(scale=scale, size=n)
        pa_size = X.shape[1]
        if pa_size == 0:
            return z
        if sem_type == 'mlp':
            hidden = 100
            W1 = np.random.uniform(low=0.5, high=2.0, size=[pa_size, hidden])
            W1[np.random.rand(*W1.shape) < 0.5] *= -1
            W2 = np.random.uniform(low=0.5, high=2.0, size=hidden)
            W2[np.random.rand(hidden) < 0.5] *= -1
            x = sigmoid(X @ W1) @ W2 + z
        elif sem_type == 'mim':
            w1 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
            w1[np.random.rand(pa_size) < 0.5] *= -1
            w2 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
            w2[np.random.rand(pa_size) < 0.5] *= -1
            w3 = np.random.uniform(low=0.5, high=2.0, size=pa_size)
            w3[np.random.rand(pa_size) < 0.5] *= -1
            x = np.tanh(X @ w1) + np.cos(X @ w2) + np.sin(X @ w3) + z
        elif sem_type == 'gp':
            from sklearn.gaussian_process import GaussianProcessRegressor
            gp = GaussianProcessRegressor()
            x = gp.sample_y(X, random_state=None).flatten() + z
        elif sem_type == 'gp-add':
            from sklearn.gaussian_process import GaussianProcessRegressor
            gp = GaussianProcessRegressor()
            x = sum([gp.sample_y(X[:, i, None], random_state=None).flatten()
                     for i in range(X.shape[1])]) + z
        else:
            raise ValueError('unknown sem type')
        return x

    d = B.shape[0]
    scale_vec = noise_scale if noise_scale else np.ones(d)
    X = np.zeros([n, d])
    G = ig.Graph.Adjacency(B.tolist())
    ordered_vertices = G.topological_sorting()
    assert len(ordered_vertices) == d
    for j in ordered_vertices:
        parents = G.neighbors(j, mode=ig.IN)
        X[:, j] = _simulate_single_equation(X[:, parents], scale_vec[j])
    return X



Evaluation metric between the estimated graph and the ground truth graph

In [12]:
g_dag, adj_dag = random_dag_generation(20, 0.1, 'er')
X = generate_single_dataset(g_dag, 100, 'gauss', 1)

In [13]:
np.savetxt('W_true.csv', adj_dag, delimiter=',')
np.savetxt('X.csv', X, delimiter=',')
W_est = notears_linear(X, lambda1=0.1, loss_type='l2')
# assert is_dag(W_est)
np.savetxt('W_est.csv', W_est, delimiter=',')

In [14]:
acc = count_accuracy(adj_dag, W_est != 0)
print(acc)

{'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'structural_hamming_du': 0, 'nnz': 15}


In [7]:
n, d, s0, graph_type, sem_type = 100, 20, 20, 'ER', 'gauss'
B_true = simulate_dag(d, s0, graph_type)
W_true = simulate_parameter(B_true)
np.savetxt('W_true.csv', W_true, delimiter=',')

X = simulate_linear_sem(W_true, n, sem_type)
np.savetxt('X.csv', X, delimiter=',')

W_est = notears_linear(X, lambda1=0.1, loss_type='l2')
assert is_dag(W_est)
np.savetxt('W_est.csv', W_est, delimiter=',')
acc = count_accuracy(B_true, W_est != 0)
print(acc)

NameError: name 'ig' is not defined

**GraN-DAG**

Similar approach but adapted to non linear structural equation models. Here, the data generating process for variable Xj is:
$$
X_j = f_j (X_j) + \epsilon_j
$$
where $\epsilon_j$ is a noise term, and $f_j$ is a non-linear function that depends on a parameter $\theta_j \in \mathbb{R}^m$.
To learn each function $f_j$, we train a neural network for each variable $X_j$. Each neural network's output matches the dimension of $\theta_j$ and the procedure to get a weighted adjacency matrix is described in section 3 of the GranDAG paper.
Below is provided the "base" model for training the j neural networks